In [5]:

from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
import numpy as np
import random
import sys

import numpy as np
import csv


synthetic_data_set = "syntheticDetailed/naive_c5_q50_s4000_v0.csv"

Using Theano backend.


In [11]:
# Read in the data set
# This function can be moved to utils.py
data_array = np.array(list(csv.reader(open(synthetic_data_set,"rb"),delimiter=','))).astype('int')
print (data_array[0:10])
num_samples = data_array.shape[0]
num_problems = data_array.shape[1]

# Split data into train and test (half and half)
train = data_array[0:num_samples/2,:]
test = data_array[num_samples/2:num_samples,:]
print (train.shape)
print (test.shape)

num_train = train.shape[0]
num_test = test.shape[0]

print('Vectorization...')
X_train = np.zeros((num_train, num_problems, num_problems * 2), dtype=np.bool)
y_train = np.zeros((num_train, num_problems), dtype=np.bool)

# Create 3-dimensional input tensor with one-hot encodings for each sample
# the dimension of each vector for a student i and time t is 2 * num_problems
# where the first half corresponds to the correctly answered problems and the
# second half to the incorrectly answered ones.
for i in xrange(num_train):
    for t xrange(num_problems):
        if train[i,t] == 1:
            X_train[i, t, t] = 1
        else:
            X_train[i, t, num_problems + t] = 1
        y_train[i, t] = train


# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(num_train, num_problems * 2)))
model.add(Dropout(0.2))
# model.add(LSTM(512, return_sequences=False))
# model.add(Dropout(0.2))
model.add(Dense(num_problems))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')



[[0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0
  0 1 1 0 1 1 0 1 0 1 1 0 1]
 [1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
  1 0 0 0 1 0 1 0 0 1 0 0 0]
 [0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 1 1 1 0 0 1 1 1]
 [0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1
  0 1 0 0 1 1 1 1 0 0 1 1 1]
 [0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 0 1 0 1 1 1 1 1]
 [0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1
  1 1 0 0 0 0 1 0 0 0 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 0
  1 0 1 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1
  1 0 1 0 0 0 1 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1